In [1]:
import os
import json
import GEOparse
import pandas as pd
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
gse= GEOparse.get_GEO(geo="GSE12277", destdir="./data/GEO")

04-Mar-2025 10:10:33 DEBUG utils - Directory ./data/GEO already exists. Skipping.
04-Mar-2025 10:10:33 INFO GEOparse - File already exist: using local version.
04-Mar-2025 10:10:33 INFO GEOparse - Parsing ./data/GEO/GSE12277_family.soft.gz: 
04-Mar-2025 10:10:33 DEBUG GEOparse - DATABASE: GeoMiame
04-Mar-2025 10:10:33 DEBUG GEOparse - SERIES: GSE12277
04-Mar-2025 10:10:33 DEBUG GEOparse - PLATFORM: GPL570
/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/venv/lib/python3.12/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
04-Mar-2025 10:10:38 DEBUG GEOparse - SAMPLE: GSM308237
04-Mar-2025 10:10:38 DEBUG GEOparse - SAMPLE: GSM308239
04-Mar-2025 10:10:38 DEBUG GEOparse - SAMPLE: GSM308241
04-Mar-2025 10:10:39 DEBUG GEOparse - SAMPLE: GSM308242
04-Mar-2025 10:10:39 DEBUG GEOparse - SAMPLE: GSM308244
04-Mar-2025 10:10:40 DEBU

### Predicate generation for table entries

In [7]:
gsm= gse.gsms["GSM308246"]
data= gsm.table
data.head()

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,AFFX-BioB-5_at,115.4570,P,0.000581
1,AFFX-BioB-M_at,119.7700,P,0.000044
2,AFFX-BioB-3_at,84.8163,P,0.000095
3,AFFX-BioC-5_at,296.3510,P,0.000052
4,AFFX-BioC-3_at,470.6520,P,0.000044


In [8]:
gsm_id = gsm.metadata.get("geo_accession", ["Unknown_GSE"])[0]
print(gsm_id)

GSM308246


In [9]:
sample_data=data.sample(100)
print(sample_data) 

            ID_REF       VALUE ABS_CALL  DETECTION P-VALUE
27719    227852_at   86.841300        P           0.001221
8943   209389_x_at  300.885000        P           0.001953
5651   206064_s_at    6.222890        A           0.500000
3243     203655_at   69.555400        P           0.000732
31631    231764_at  104.658000        P           0.000732
...            ...         ...      ...                ...
45562   1553338_at   31.829100        P           0.014160
44761   1552266_at    3.959370        A           0.246094
6091     206504_at    3.781950        P           0.030273
37729  237867_s_at    0.994216        A           0.665527
37354    237492_at    5.460560        A           0.194580

[100 rows x 4 columns]


In [10]:
columns = list(sample_data.columns)
print("Extracted Columns:", columns)


Extracted Columns: ['ID_REF', 'VALUE', 'ABS_CALL', 'DETECTION P-VALUE']


In [11]:
prompt = f"""
    You are a data standardization expert. Convert the following dataset column names into standardized predicate names:
    {columns}
    Return a JSON dictionary where keys are original column names and values are standardized predicate names.
    """

In [12]:
def openai_generate(messages: list, model: str = 'gpt-4', **generation_params):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    completion = client.chat.completions.create(
        messages=messages,
        model=model,
        **generation_params
    )
    return completion

In [13]:
messages =[
    {'role':'system', 'content':"You are an expert in data standardization."},
    {'role':'user', 'content':prompt}
]

In [14]:
completion = openai_generate(messages, temperature=1.0)
predicate_mapping=completion.choices[0].message.content
print("Predicate Mapping:", predicate_mapping)

Predicate Mapping: {
    "ID_REF": "id_ref",
    "VALUE": "value",
    "ABS_CALL": "abs_call",
    "DETECTION P-VALUE": "detection_p_value"
}


In [15]:
predicate_mapping = json.loads(predicate_mapping)

In [16]:
def generate_predicate_entry(row, predicate_mapping, gsm_id):
    predicate_values = [f'gsmId("{gsm_id}")']  # Add gsmId predicate
    for original_col, predicate in predicate_mapping.items():
        value = row.get(original_col, "null")
        predicate_values.append(f'{predicate}({value if pd.notna(value) else "null"})')
    return f'GSM_Measurement(\n    {",\n    ".join(predicate_values)}\n)'


In [17]:
predicate_entries = sample_data.apply(lambda row: generate_predicate_entry(row, predicate_mapping, gsm_id), axis=1)

print("Example Predicate Output:")
print("\n".join(predicate_entries))



Example Predicate Output:
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(227852_at),
    value(86.8413),
    abs_call(P),
    detection_p_value(0.0012207)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(209389_x_at),
    value(300.885),
    abs_call(P),
    detection_p_value(0.00195313)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(206064_s_at),
    value(6.22289),
    abs_call(A),
    detection_p_value(0.5)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(203655_at),
    value(69.5554),
    abs_call(P),
    detection_p_value(0.000732422)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(231764_at),
    value(104.658),
    abs_call(P),
    detection_p_value(0.000732422)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(224645_at),
    value(75.764),
    abs_call(P),
    detection_p_value(0.000732422)
)
GSM_Measurement(
    gsmId("GSM308246"),
    id_ref(209401_s_at),
    value(4.7021),
    abs_call(A),
    detection_p_value(0.213379)
)
GSM_Measurement(
 

In [18]:
with open("predicates.txt", "w") as file:
    file.write("\n".join(predicate_entries))

### Generating predicates that represent the relationship between GSE and GSM

In [83]:
gse.metadata

{'title': ['Hematopoietic Progenitor Cells of Different Donor Age'],
 'geo_accession': ['GSE12277'],
 'status': ['Public on Jun 12 2009'],
 'submission_date': ['Jul 29 2008'],
 'last_update_date': ['Mar 25 2019'],
 'pubmed_id': ['19513108'],
 'summary': ['In this series we have analyzed the effect of donor age on the gene expression profile of human hematopoietic stem and progenitor cells (HPC). Cells were taken from umbilical cord blood (CB) or from G-CSF mobilized blood of healthy donors for allogeneic blood stem cell transplantation.'],
 'overall_design': ['Hematopoietic progenitor cells (HPC) were isolated within the CD34+ cell fraction from fresh human cord blood (CB) or from G-CSF mobilized peripheral blood (PB) as described in detail before (Wagner et al., Blood, 2004, 104:675-684; Wagner et al., Stem cells, 2005, 23:1180-1191; Wagner et al., 2007, 10:2638-2657). Differential gene expression was subsequently compared in the CD34+ samples of different donor age.'],
 'type': ['Exp

In [84]:
def generate_fol_relationship(metadata):
    """
    Generates First-Order Logic (FOL) representation of the GSE-GSM relationship.

    Args:
        metadata (dict): Dictionary containing GSE metadata.

    Returns:
        str: FOL statements representing GSE-GSM relationships.
    """
    gse_id = metadata.get("geo_accession", ["Unknown_GSE"])[0]  # Get GSE ID
    gsm_ids = metadata.get("sample_id", [])  # Get GSM IDs list

    fol_statements = [f"has_sample({gse_id}, {gsm});" for gsm in gsm_ids]  # Create FOL statements

    return "\n".join(fol_statements)

In [85]:
gse_metadata= gse.metadata

In [86]:
fol_output = generate_fol_relationship(gse_metadata)

In [87]:
print("Example FOL Output:")
print(fol_output)

Example FOL Output:
has_sample(GSE12277, GSM308237);
has_sample(GSE12277, GSM308239);
has_sample(GSE12277, GSM308241);
has_sample(GSE12277, GSM308242);
has_sample(GSE12277, GSM308244);
has_sample(GSE12277, GSM308245);
has_sample(GSE12277, GSM308246);
has_sample(GSE12277, GSM308247);
has_sample(GSE12277, GSM308254);
has_sample(GSE12277, GSM308255);
has_sample(GSE12277, GSM308317);
has_sample(GSE12277, GSM308461);
has_sample(GSE12277, GSM308462);
has_sample(GSE12277, GSM308463);
has_sample(GSE12277, GSM308465);
has_sample(GSE12277, GSM308467);
has_sample(GSE12277, GSM308469);
has_sample(GSE12277, GSM308470);
has_sample(GSE12277, GSM308473);
